In [1]:
import numpy as np
import keras
from keras import layers
import tensorflow as tf
from tensorflow import data
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as clrs

In [2]:
'''
Creates the 3 Models to be tested
'''
def createModel(modelNum, input_shape, activation_function, num_classes):
  m1 = keras.Sequential(
            [
                keras.Input(shape=input_shape),
                layers.Conv2D(32, kernel_size=(3, 3), activation=activation_function),
                layers.MaxPooling2D(pool_size=(2, 2)),
                layers.Conv2D(64, kernel_size=(3, 3), activation=activation_function),
                layers.MaxPooling2D(pool_size=(2, 2)),
                layers.Flatten(),
                layers.Dropout(0.5),
                layers.Dense(num_classes, activation="softmax"),
            ]
        )
  m2 = keras.Sequential(
            [
                keras.Input(shape=input_shape),
                layers.Conv2D(32, kernel_size=(3, 3), activation=activation_function),
                layers.MaxPooling2D(pool_size=(2, 2)),
                layers.Conv2D(64, kernel_size=(3, 3), activation=activation_function),
                layers.MaxPooling2D(pool_size=(2, 2)),
                layers.Flatten(),
                layers.Dense(num_classes, activation="softmax"),
            ]
        )
  m3 = keras.Sequential(
            [
                keras.Input(shape=input_shape),
                layers.Conv2D(32, kernel_size=(3, 3), activation=activation_function),
                layers.MaxPooling2D(pool_size=(2, 2)),
                layers.Conv2D(64, kernel_size=(3, 3), activation=activation_function),
                layers.MaxPooling2D(pool_size=(2, 2)),
                layers.Conv2D(128, kernel_size=(3, 3), activation=activation_function),
                layers.MaxPooling2D(pool_size=(2, 2)),
                layers.Flatten(),
                layers.Dropout(0.5),
                layers.Dense(num_classes, activation="softmax"),
            ]
        )

  if modelNum == 1:
    model= m1

  elif modelNum == 2:
    model= m2
  else:
    model= m3

  return model

In [3]:
#splitRatios= [0.8, 0.85, 0.9]
#learningRates= [0.01, 0.001, 0.0001]
#activationFunctions = ["relu", "linear", "mish"]
splitRatios=[0.85]
learningRates=[0.001]
activationFunctions=["relu"]
modelNums=[1,2,3]

accuracies = []
losses= []
precisions=[]
recalls=[]
f1Scores=[]
params=[]

for m in modelNums:
  for split_ratio in splitRatios:

    # Model / data parameters
    num_classes = 10
    input_shape = (28, 28, 1)

    # Load the data and split it between train and test sets
    (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

    # Scale images to the [0, 1] range
    x_train = x_train.astype("float32") / 255
    x_test = x_test.astype("float32") / 255
    # Make sure images have shape (28, 28, 1)
    x_train = np.expand_dims(x_train, -1)
    x_test = np.expand_dims(x_test, -1)

    #Resizing test/train sets
    np.random.seed(2019)
    x = np.concatenate((x_train, x_test))
    y = np.concatenate((y_train, y_test))
    index = np.random.rand(len(x)) < split_ratio # boolean index
    x_train, x_test = x[index], x[~index]  # index and it's negation
    y_train, y_test = y[index], y[~index]


    #print("x_train shape:", x_train.shape)
    #print(x_train.shape[0], "train samples")
    #print(x_test.shape[0], "test samples")


    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)

    for learning_rate in learningRates:
      for activation_function in activationFunctions:
        p= [m, split_ratio, learning_rate, activation_function]
        params.append(p)

        print("***********************************************************************************************************************")
        #print("Test Number:",str(ct))
        print("Model Number:" ,str(m))
        print("Split Ratio:", str(split_ratio))
        print("Learning Rate:",str(learning_rate))
        print("Activation Function:" ,str(activation_function))
        #insert model code here
        model= createModel(m,input_shape, activation_function, num_classes)
        model.summary()

        batch_size = 128
        epochs = 15
        optimizer = tf.keras.optimizers.Adam(learning_rate)
        model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy", "Precision", "Recall", "F1Score"])

        model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, verbose=0)

        score = model.evaluate(x_test, y_test, verbose=0)

        print("Test Loss:", score[0])
        print("Test Accuracy:", score[1])
        print("Test Precision:", score[2])
        print("Test Recalls:", score[3])
        print("Test F1 Score:",score[4])

        accuracies.append(score[1])
        losses.append(score[0])
        precisions.append(score[2])
        recalls.append(score[3])
        f1Scores.append(score[4])

11490434/11490434 [==============================] - 0s 0us/step
***********************************************************************************************************************
Model Number: 1
Split Ratio: 0.85
Learning Rate: 0.001
Activation Function: relu
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open('/content/drive/MyDrive/projectResultsM4.txt', mode='wt', encoding='utf-8') as myfile:
    for a in range(len(accuracies)):
        myfile.write(str(params[a]) + " " + str(accuracies[a]) + " " + str(losses[a]) + " " + str(precisions[a]) + " " + str(recalls[a]) + " " + str(f1Scores[a])+ "\n")

In [ ]:
import csv

with open('/content/drive/MyDrive/projectResults.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    field = ["Test Number", "Model Number", "Split Ratio", "Learning Rate", "Activation Function", "Accuracy", "Loss", "Precision", "Recall", "F1 Score"]
    writer.writerow(field)
    for x in range(len(accuracies)):
      writer.writerow([x, params[x][0],params[x][1],params[x][2],params[x][3],accuracies[x], losses[x], precisions[x], recalls[x], f1Scores[x]])

NameError: ignored

In [ ]:
maxAcc=0
maxIndx=0
for x in range(len(accuracies)):
  if accuracies[x]> maxAcc:
    maxAcc= accuracies[x]
    maxIndx= x

print("Max Accuracy: ", maxAcc)
print("Index: ", maxIndx)
print("Params: ", params[maxIndx])

Max Accuracy:  0.9908046126365662
Index:  77
Params:  ['3' '0.9' '0.001' 'mish']


In [ ]:
bestLoss=100
bestIndx=0
for x in range(len(losses)):
  if losses[x]<bestLoss:
    bestLoss= losses[x]
    bestIndx= x

print("Min Loss: ", bestLoss)
print("Index: ", bestIndx)
print("Params: ", params[bestIndx])

Min Loss:  0.03123272955417633
Index:  23
Params:  ['1' '0.9' '0.001' 'mish']


In [ ]:
maxPre=0
maxIndx=0
for x in range(len(precisions)):
  if precisions[x]> maxPre:
    maxPre= precisions[x]
    maxIndx= x

print("Max Precision: ", maxPre)
print("Index: ", maxIndx)
print("Params: ", params[maxIndx])

Max Precision:  0.9912167191505432
Index:  21
Params:  ['1' '0.9' '0.001' 'relu']


In [ ]:
maxRecall=0
maxIndx=0
for x in range(len(recalls)):
  if recalls[x]> maxRecall:
    maxRecall= recalls[x]
    maxIndx= x

print("Max Recall: ", maxRecall)
print("Index: ", maxIndx)
print("Params: ", params[maxIndx])

Max Recall:  0.9903735518455505
Index:  77
Params:  ['3' '0.9' '0.001' 'mish']


In [ ]:
m1Accuracies= np.array(accuracies[0:27])
m2Accuracies= np.array(accuracies[27:54])
m3Accuracies= np.array(accuracies[54:81])

m1Losses= np.array(losses[0:27])
m2Losses= np.array(losses[27:54])
m3Losses= np.array(losses[54:81])

m1Precisions= np.array(precisions[0:27])
m2Precisions= np.array(precisions[27:54])
m3Precisions= np.array(precisions[54:81])

m1Recalls= np.array(recalls[0:27])
m2Recalls= np.array(recalls[27:54])
m3Recalls= np.array(recalls[54:81])

In [ ]:
m1ReluAcc= np.array([])
m1LinearAcc= np.array([])
m1MishAcc= np.array([])

for x in range(len(m1Accuracies)):
  if x%3 ==0:
    m1ReluAcc= np.append(m1ReluAcc, m1Accuracies[x])
  elif x%3 ==1:
    m1LinearAcc= np.append(m1LinearAcc, m1Accuracies[x])
  else:
    m1MishAcc= np.append(m1MishAcc, m1Accuracies[x])

m2ReluAcc= np.array([])
m2LinearAcc= np.array([])
m2MishAcc= np.array([])

for x in range(len(m2Accuracies)):
  if x%3 ==0:
    m2ReluAcc= np.append(m2ReluAcc, m2Accuracies[x])
  elif x%3 ==1:
    m2LinearAcc= np.append(m2LinearAcc, m2Accuracies[x])
  else:
    m2MishAcc= np.append(m2MishAcc, m2Accuracies[x])

m3ReluAcc= np.array([])
m3LinearAcc= np.array([])
m3MishAcc= np.array([])

for x in range(len(m3Accuracies)):
  if x%3 ==0:
    m3ReluAcc= np.append(m3ReluAcc, m3Accuracies[x])
  elif x%3 ==1:
    m3LinearAcc= np.append(m3LinearAcc, m3Accuracies[x])
  else:
    m3MishAcc= np.append(m3MishAcc, m3Accuracies[x])
